<h1 style="text-align: center;" class="list-group-item list-group-item-action active">NYC AirBnB Data Analysis</h1>
<ul style="list-style-type:none;">
    <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#1" role="tab">
            1. Introduction<span class="badge badge-primary badge-pill">1</span>
        </a>
    </li>
    <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2" role="tab">
            2. Exploratory Analysis<span class="badge badge-primary badge-pill">2</span>
        </a>
        <ul style="list-style-type:none;">
            <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.1" role="tab">
                    2.1 Setup: Imports & Plotting Functions<span class="badge badge-primary badge-pill">3</span>
                </a>
            </li>
            <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.2" role="tab">
                    2.2 Load and Inspect Data<span class="badge badge-primary badge-pill">4</span>
                </a>
            </li>
            <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.3" role="tab">
                    2.3 Data Visualization<span class="badge badge-primary badge-pill">5</span>
                </a>
            </li>
        </ul>
    </li>
    <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#3" role="tab">
            3. Conclusion<span class="badge badge-primary badge-pill">6</span>
        </a>
    </li>
</ul>


<h1 style="text-align: center" class="list-group-item list-group-item-action active">1. Introduction</h1><a id="1"></a>


Greetings from the Kaggle bot! This is an automatically-generated kernel with starter code demonstrating how to read in the data and begin exploring. Click the blue "Edit Notebook" or "Fork Notebook" button at the top of this kernel to begin editing.


<h1 style="text-align: center" class="list-group-item list-group-item-action active">2. Exploratory Analysis</h1><a id="2"></a>


To begin this exploratory analysis, first use `matplotlib` to import libraries and define functions for plotting the data. Depending on the data, not all plots will be made. (Hey, I'm just a kerneling bot, not a Kaggle Competitions Grandmaster!)


<h2 style="text-align: center" class="list-group-item list-group-item-success">2.1 Setup: Imports & Plotting Functions</h2><a id="2.1"></a>


In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


The next hidden code cells define functions for plotting data. Click on the "Code" button in the published kernel to reveal the hidden code.


In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()


In [ ]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()


In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()


<h2 style="text-align: center" class="list-group-item list-group-item-success">2.2 Load and Inspect Data</h2><a id="2.2"></a>


Now you're ready to read in the data and use the plotting functions to visualize the data.


### Let's check the `AB_NYC_2019.csv` file:


In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# AB_NYC_2019.csv has 48895 rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('/kaggle/input/AB_NYC_2019.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'AB_NYC_2019.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')


Let's take a quick look at what the data looks like:


In [ ]:
df1.head(5)


<h2 style="text-align: center" class="list-group-item list-group-item-success">2.3 Data Visualization</h2><a id="2.3"></a>


Distribution graphs (histogram/bar graph) of sampled columns:


In [ ]:
plotPerColumnDistribution(df1, 10, 5)


Correlation matrix:


In [ ]:
plotCorrelationMatrix(df1, 8)


Scatter and density plots:


In [ ]:
plotScatterMatrix(df1, 20, 10)


<h1 style="text-align: center" class="list-group-item list-group-item-action active">3. Conclusion</h1><a id="3"></a>


This concludes your starter analysis! To go forward from here, click the blue "Edit Notebook" button at the top of the kernel. This will create a copy of the code and environment for you to edit. Delete, modify, and add code as you please. Happy Kaggling!
